# Coding

In [2]:
# Your revised Python pipeline with TT parameter scaling applied correctly

# ---- Unique dir ----
identifier = 'bD_Z2_S2' 
EXP_NAME = identifier

# ---- Column names ----
METERING_COLUMN    = "Main Heating"
START_DATE = "2023-01-01"
END_DATE   = "2023-12-31"
AREA_M2 = 9184

# ---- Source files ----
SRC_METERING = "../ihg_generator/Metering.csv" 
SRC_TT = f"../eplus_diagnosis/idf-Oct/G2/auto-cal/{identifier}/samples.csv" 
SRC_Y = f"../eplus_diagnosis/{identifier}_csv/DistrictHeatingWater_Facility_timeseries.csv"
# ---- R environment ----
CALIBRO_NAME = "simplest"
RSCRIPT_BIN = "Rscript"
PARAM_TO_SCALE = "a5_infiltration_rate"
SCALE_FACTOR = 3600

# ---- Fixed term ----
METERING_TIMESTAMP = "Timestamp"
TT_DROP_COLUMNS = ["CSV File"]

In [3]:
import pandas as pd
from pathlib import Path
import subprocess
import sys

# ================================================================
# 0. Resolve experiment directories
# ================================================================
base_dir = Path(".").resolve()
exp_dir = base_dir / EXP_NAME

data_dir  = exp_dir / "data"
code_dir  = exp_dir / "code"
output_dir = exp_dir / "output"

for d in [data_dir, code_dir, output_dir]:
    d.mkdir(parents=True, exist_ok=True)

print(f"[INFO] Experiment folder prepared at: {exp_dir}")

# ================================================================
# KEY: Extract TT.csv + rescale chosen parameter
# ================================================================
df_TT = pd.read_csv(SRC_TT)

# Drop unused columns
for col in TT_DROP_COLUMNS:
    if col in df_TT.columns:
        df_TT = df_TT.drop(columns=[col])

# ---- RESCALE ONE PARAMETER ----
if PARAM_TO_SCALE in df_TT.columns:
    df_TT[PARAM_TO_SCALE] = df_TT[PARAM_TO_SCALE] * SCALE_FACTOR
    print(f"[INFO] Scaled parameter '{PARAM_TO_SCALE}' by ×{SCALE_FACTOR}")
else:
    print(f"[WARNING] Parameter '{PARAM_TO_SCALE}' not found in TT columns. Available columns: {list(df_TT.columns)}")

# Save TT
df_TT.to_csv(data_dir / "TT.csv", index=False)
print("[INFO] TT.csv saved:", df_TT.shape)



[INFO] Experiment folder prepared at: /Users/rui.bo/Desktop/Working/1-phd_mainworks/Y3/calibro/bD_Z2_S2
[INFO] Scaled parameter 'a5_infiltration_rate' by ×3600
[INFO] TT.csv saved: (100, 10)


In [7]:
import pandas as pd
from pathlib import Path

# ================================================================
# Helpers
# ================================================================

def _aggregate_monthly(df, timestamp_col, value_cols):
    df[timestamp_col] = pd.to_datetime(df[timestamp_col])
    monthly = (
        df.set_index(timestamp_col)[value_cols]
        .resample("M")
        .sum()
        .reset_index(drop=True)
    )
    return monthly

# ================================================================
# 1. Extract obs.csv (hourly or monthly)
# ================================================================

def extract_obs(
    SRC_METERING,
    START_DATE,
    END_DATE,
    METERING_TIMESTAMP,
    METERING_COLUMN,
    AREA_M2,
    data_dir: Path,
    mode="hourly",
):
    df_meter = pd.read_csv(SRC_METERING)
    df_meter[METERING_TIMESTAMP] = pd.to_datetime(df_meter[METERING_TIMESTAMP], dayfirst=True)

    df_obs = df_meter.loc[
        (df_meter[METERING_TIMESTAMP] >= START_DATE)
        & (df_meter[METERING_TIMESTAMP] < END_DATE),
        [METERING_TIMESTAMP, METERING_COLUMN],
    ].copy()

    df_obs[METERING_COLUMN] = df_obs[METERING_COLUMN] / AREA_M2

    if mode == "monthly":
        df_obs = _aggregate_monthly(df_obs, METERING_TIMESTAMP, [METERING_COLUMN])

    df_out = df_obs[[METERING_COLUMN]]
    df_out.to_csv(data_dir / "obs.csv", index=False)
    print("[INFO] obs.csv saved:", df_out.shape)
    return df_out

# ================================================================
# 2. Extract Y.csv (hourly or monthly)
# ================================================================

def extract_Y(
    SRC_Y,
    START_DATE,
    END_DATE,
    data_dir: Path,
    mode="hourly",
):
    df_Y = pd.read_csv(SRC_Y)

    ts_cols = [c for c in df_Y.columns if "time" in c.lower() or "stamp" in c.lower()]
    if not ts_cols:
        raise ValueError("No timestamp column found in Y file")
    ts_col = ts_cols[0]

    df_Y[ts_col] = pd.to_datetime(df_Y[ts_col])

    df_Y_filtered = df_Y.loc[
        (df_Y[ts_col] >= START_DATE)
        & (df_Y[ts_col] < END_DATE)
    ].copy()

    if mode == "monthly":
        value_cols = [c for c in df_Y.columns if c != ts_col]
        df_Y_filtered = _aggregate_monthly(df_Y_filtered, ts_col, value_cols)

    df_Y_final = df_Y_filtered.drop(columns=[ts_col], errors="ignore")

    # NEW: sort Y columns alphabetically
    df_Y_final = df_Y_final.reindex(sorted(df_Y_final.columns), axis=1)

    df_Y_final.to_csv(data_dir / "Y.csv", index=False)
    print("[INFO] Y.csv saved:", df_Y_final.shape)
    return df_Y_final

# ================================================================
# Validation wrapper
# ================================================================

def run_and_validate(SRC_METERING, SRC_Y, START_DATE, END_DATE,
                     METERING_TIMESTAMP, METERING_COLUMN, AREA_M2,
                     data_dir: Path, mode="hourly"):

    df_obs = extract_obs(
        SRC_METERING,
        START_DATE,
        END_DATE,
        METERING_TIMESTAMP,
        METERING_COLUMN,
        AREA_M2,
        data_dir,
        mode=mode,
    )

    df_Y_final = extract_Y(
        SRC_Y,
        START_DATE,
        END_DATE,
        data_dir,
        mode=mode,
    )

    if df_obs.shape[0] != df_Y_final.shape[0]:
        raise ValueError("❌ obs and Y row count mismatch!")

    print("✓ Shape validation passed.")
run_and_validate(SRC_METERING, SRC_Y, START_DATE, END_DATE,
                     METERING_TIMESTAMP, METERING_COLUMN, AREA_M2,
                     data_dir, mode="monthly")

[INFO] obs.csv saved: (12, 1)
[INFO] Y.csv saved: (12, 100)
✓ Shape validation passed.


/var/folders/7p/_yzdn45s5c7_bd88lqmn1cw80000gn/T/ipykernel_65619/2669456937.py:12: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample("M")
/var/folders/7p/_yzdn45s5c7_bd88lqmn1cw80000gn/T/ipykernel_65619/2669456937.py:12: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample("M")


In [8]:

# ================================================================
# 5. Generate R script
# ================================================================
r_script_path = code_dir / "run_calibro.R"

r_code = f"""
setwd("{output_dir}")

library(calibro)

CE <- calEnv$new(name = '{CALIBRO_NAME}')

CE$add.ds(
    name = 'data1',
    Y.star = '{data_dir}/obs.csv',
    TT     = '{data_dir}/TT.csv',
    Y      = '{data_dir}/Y.csv'
)

CE$rd = 'pca'
CE$sa = 'sobolSmthSpl'
CE$ret = list(mthd = 'ng.screening')
CE$mdls = 'gpr.ng.sePar01_whitePar01'
CE$train = list(type = 'training', alg = 'amoeba')
CE$cals = 'cal.gpr.ng'
CE$cal.mcmc = list(alg = 'amg')

CE$cal.res()

# IMPORTANT: no 'path' argument allowed
CE$genReport(
    type = 'pdf',
    out  = c('dss','cal')
)
"""

with open(r_script_path, "w") as f:
    f.write(r_code)

print("[INFO] R script generated:", r_script_path)


# ================================================================
# 6. Run Rscript
# ================================================================
print("\n[INFO] Running Calibro Rscript...")
cmd = [RSCRIPT_BIN, str(r_script_path)]
result = subprocess.run(cmd)

if result.returncode == 0:
    print("✓ Calibro finished successfully.")
else:
    print("❌ Calibro failed.")


print(f"\n🎉 Experiment complete. Results saved to: {output_dir}")

[INFO] R script generated: /Users/rui.bo/Desktop/Working/1-phd_mainworks/Y3/calibro/bD_Z2_S2/code/run_calibro.R

[INFO] Running Calibro Rscript...


Loading required package: R6
Loading required package: coda
Loading required package: parallel
Loading required package: MASS
Loading required package: modeest



### NELDER-MEAD DOWNHILL SIMPLEX ALGORITHM ###

 Version: amoeba
 Start time: 2025-11-17 15:50:04.779999 
 Number of simulations: 5 


*** Simulation 1 / 5 (elapsed time: 0.00247 mins) ***

 best y: 648.662407859101
 best z:
	 %RHO@d1_htgsp_office%@sePar01.pc.mu = 0.009293 
	 %RHO@d2_htgsp_office_st%@sePar01.pc.mu = 0.9813 
	 %RHO@e1_natural_ventilation_rate%@sePar01.pc.mu = 1 
	 %ALPHA1@sePar01.pc.mu = 1 
	 %SIGMA1@wn1.pc.mu = 0.9998 
	 %SIGMA1@wn2.pc.mu = 27470000 

*** Simulation 2 / 5 (elapsed time: 0.00639 mins) ***

 best y: 779.796405514213
 best z:
	 %RHO@d1_htgsp_office%@sePar01.pc.mu = 0.7809 
	 %RHO@d2_htgsp_office_st%@sePar01.pc.mu = 0.8039 
	 %RHO@e1_natural_ventilation_rate%@sePar01.pc.mu = 0.3854 
	 %ALPHA1@sePar01.pc.mu = 1 
	 %SIGMA1@wn1.pc.mu = 1 
	 %SIGMA1@wn2.pc.mu = 25870000 

*** Simulation 3 / 5 (elapsed time: 0.00828 mins) ***

 best y: 779.796405514213
 best z:
	 %RHO@d1_htgsp_office%@sePar01.pc.mu = 0.7809 
	 %RHO@d2_htgsp_office_st%@sePar01.pc.mu = 0.8039 
	

Warning messages:
1: encountered a tie, and the difference between minimal and 
                   maximal value is > length('x') * 'tie.limit'
the distribution could be multimodal 
2: encountered a tie, and the difference between minimal and 
                   maximal value is > length('x') * 'tie.limit'
the distribution could be multimodal 
3: encountered a tie, and the difference between minimal and 
                   maximal value is > length('x') * 'tie.limit'
the distribution could be multimodal 
4: encountered a tie, and the difference between minimal and 
                   maximal value is > length('x') * 'tie.limit'
the distribution could be multimodal 


processing file: /opt/homebrew/lib/R/4.3/site-library/calibro/calibro_report.Rnw


1/44             
2/44 [setup]     
3/44             
4/44 [exeSumm]   
5/44             
6/44 [content]   
7/44             
8/44 [genVar]    
9/44             
10/44 [dsVar]     
11/44 [dsText]    
12/44             
13/44 [ds1]       
14/44 [ds2]       
15/44 [ds3]       
16/44 [ds4]       
17/44 [ds5]       
18/44 [ds6]       
19/44 [ds7]       
20/44 [ds8]       
21/44 [ds9]       
22/44 [ds10]      
23/44             
24/44 [saVar]     
25/44 [saText]    
26/44             
27/44 [ret_tables]
28/44             
29/44 [train]     
30/44             
31/44 [calVar]    
32/44 [cal]       
33/44 [thetaplot] 
34/44 [ds1fit]    
35/44 [ds2fit]    
36/44 [ds3fit]    
37/44 [ds4fit]    
38/44 [ds5fit]    
39/44 [ds6fit]    
40/44 [ds7fit]    
41/44 [ds8fit]    
42/44 [ds9fit]    
43/44 [ds10fit]   
44/44             


output file: calibro_report.tex



[1] "calibro_report.pdf"
✓ Calibro finished successfully.

🎉 Experiment complete. Results saved to: /Users/rui.bo/Desktop/Working/1-phd_mainworks/Y3/calibro/bD_Z2_S2/output


Warning messages:
1: encountered a tie, and the difference between minimal and 
                   maximal value is > length('x') * 'tie.limit'
the distribution could be multimodal 
2: encountered a tie, and the difference between minimal and 
                   maximal value is > length('x') * 'tie.limit'
the distribution could be multimodal 
3: encountered a tie, and the difference between minimal and 
                   maximal value is > length('x') * 'tie.limit'
the distribution could be multimodal 
4: encountered a tie, and the difference between minimal and 
                   maximal value is > length('x') * 'tie.limit'
the distribution could be multimodal 
